In [1]:
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
print(keras.__version__)
import cv2  # for image processing
from sklearn.model_selection import train_test_split
import scipy.io
import os
import h5py

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


1.8.0
2.1.6


In [2]:
# !!! remember to clear session/graph if you rebuild your graph to avoid out-of-memory errors !!!
def reset_tf_session():
    K.clear_session()
    tf.reset_default_graph()
    s = K.get_session()
    return s

In [3]:
# we will crop and resize input images to IMG_SIZE x IMG_SIZE
IMG_SIZE = 256
N_CLASSES = 1

# Prepare for training

In [4]:
def load_dataset():
    train_dataset = h5py.File('./img_data.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    return train_set_x_orig, train_set_y_orig

In [5]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [6]:
# batch generator
BATCH_SIZE = 32

def batch_generator(items, labels, batch_size):
    """
    Implement batch generator that yields items in batches of size batch_size.
    There's no need to shuffle input items, just chop them into batches.
    Remember about the last batch that can be smaller than batch_size!
    Input: any iterable (list, generator, ...). You should do `for item in items: ...`
        In case of generator you can pass through your items only once!
    Output: In output yield each batch as a list of items.
    """
    
    ### YOUR CODE HERE
    m = items.shape[0]
    batch_items = []
    batch_labels = []
    for i in np.arange(m):
        if len(batch_items) < batch_size - 1:
            batch_items.append(items[i])
            batch_labels.append(labels[i])
        else:
            batch_items.append(items[i])
            batch_labels.append(labels[i])
            yield batch_items, batch_labels
            batch_items = []
            batch_labels = []
    yield batch_items, batch_labels


In [7]:
def train_generator(items, labels):
    while True:  # so that Keras can loop through this as long as it wants
        for batch_imgs, batch_labels in batch_generator(items, labels, BATCH_SIZE):
            # stack images into 4D tensor [batch_size, img_size, img_size, 3]
            batch_imgs = np.stack(batch_imgs, axis=0)
            # convert targets into 2D tensor [batch_size, num_classes]
            batch_targets = np.array(batch_labels)
            yield batch_imgs, batch_targets

In [8]:
train_X_orig, train_y_orig = load_dataset()

In [9]:
train_X = train_X_orig/255 - 0.5

In [10]:
train_X_orig.shape

(2561, 256, 256, 3)

In [11]:
train_y_orig.shape

(2561,)

In [ ]:
len(np.argwhere(train_y_orig>0.0))

## Plot an image

In [12]:
h5_img = train_X_orig[1842]
file = 'test.jpg'
cv2.imwrite(file, h5_img)

True

In [ ]:
import matplotlib.pylab as plb

plb.imshow(cv2.imread(file))

# Training

In [13]:
# remember to clear session if you start building graph from scratch!
s = reset_tf_session()
# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)

In [15]:
def inception(use_imagenet=True):
    # load pre-trained model graph, don't add final layer
    model = keras.applications.VGG16(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                          weights='imagenet' if use_imagenet else None)
    # add global pooling just like in InceptionV3
    new_output = keras.layers.GlobalAveragePooling2D()(model.output)
    # add new dense layer for our labels
    new_output = keras.layers.Dense(N_CLASSES, activation='softmax')(new_output)
    model = keras.engine.training.Model(model.inputs, new_output)
    return model

In [16]:
model = inception()

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [18]:
# set all layers trainable by default
for layer in model.layers:
    layer.trainable = True
    
# fix deep layers (fine-tuning the first layers , where are the styles)
for layer in model.layers[-100:]:
    layer.trainable = False

In [19]:
# compile new model
model.compile(
    loss='binary_crossentropy',  # we train 102-way classification
    optimizer=keras.optimizers.adamax(lr=1e-2),  # we can take big lr here because we fixed first layers
    metrics=['accuracy']  # report accuracy during training
)

In [20]:
# we will save model checkpoints to continue training in case of kernel death
model_filename = 'berni.{0:03d}.hdf5'
last_finished_epoch = None

#### uncomment below to continue training from model checkpoint
#### fill `last_finished_epoch` with your latest finished epoch
# from keras.models import load_model
# s = reset_tf_session()
# last_finished_epoch = 10
# model = load_model(model_filename.format(last_finished_epoch))

In [ ]:
model.fit(x=train_X, y=train_y_orig, epochs = 20, batch_size = 32, validation_data=(train_X, train_y_orig))

In [22]:
model.predict(train_X_orig[1841:1848,:,:,:])

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [21]:
# fine tune for 2 epochs (full passes through all training data)
# we make 2*8 epochs, where epoch is 1/8 of our training data to see progress more often
model.fit_generator(
    train_generator(train_X, train_y_orig), 
    steps_per_epoch=len(train_X) // BATCH_SIZE // 8,
    epochs=2 * 8,
    validation_data=train_generator(train_X, train_y_orig), 
    validation_steps=len(train_X) // BATCH_SIZE // 4,
    initial_epoch=last_finished_epoch or 0
)

Epoch 1/16
10/10 [==============================] - 1389s 139s/step - loss: 15.8427 - acc: 0.0063 - val_loss: 15.8178 - val_acc: 0.0078
Epoch 2/16
10/10 [==============================] - 1305s 131s/step - loss: 15.7929 - acc: 0.0094 - val_loss: 15.8677 - val_acc: 0.0047
Epoch 3/16
10/10 [==============================] - 1217s 122s/step - loss: 15.6435 - acc: 0.0187 - val_loss: 15.7680 - val_acc: 0.0109
Epoch 4/16
10/10 [==============================] - 1179s 118s/step - loss: 15.8427 - acc: 0.0063 - val_loss: 15.8178 - val_acc: 0.0078
Epoch 5/16
10/10 [==============================] - 1141s 114s/step - loss: 15.8926 - acc: 0.0031 - val_loss: 15.8427 - val_acc: 0.0063
Epoch 6/16
10/10 [==============================] - 1091s 109s/step - loss: 15.7929 - acc: 0.0094 - val_loss: 15.8377 - val_acc: 0.0066
Epoch 7/16
10/10 [==============================] - 1108s 111s/step - loss: 15.6933 - acc: 0.0156 - val_loss: 15.7182 - val_acc: 0.0141
Epoch 8/16
10/10 [==============================

In [ ]:
model.save_weights('berni_weights.h5')

In [ ]:
model.predict(train_X_orig[0:5,:,:,:])